---
toc: true
comments: true
layout: post
title: U10 Recursion P1 Student
description: Lesson on Java recursion.
type: Tangibles
courses: { csa: {week: 12} }
---

# Code For Final Project: 

I wrote a lot of code for our final project, and played a large part in both the hardware and software.

Some of my main contributions include building a lot of the frontend and backend socket. While I didn't contribute to the semaphore, I build teh original socket infrastructure that we build the rest of our project on. 

### Javascript


#### Original Architecture

```javascript
function up(){console.log("wee woo");}
function down(){console.log("wee woo");}
function left(){console.log("wee woo");} // Here, each individual function could be used to emit
function right(){console.log("wee woo");}
```

#### Second Architecture: Function-Based

Server side: The server side code ran a socket server, in which we planned to use socket functions to actually send out java API requests. 

```javascript
const express = require('express');
 const app = express();
 const http = require('http');
 const server = http.createServer(app);
 const { Server } = require("socket.io");
 const io = new Server(server);

/*
Setup:
importing dependencies and requiring in order to be utilized
Creating socket object
creating http server
*/
 var users = {};

 server.listen(3000, () => {
     console.log("listening on 3000"); // Sets socket to listen on port 3000
 });

 app.get('/', (req, res) => {
     res.sendFile(__dirname + '/index.html'); // Originally, sends idea to index.html
 });

 io.on('connection', (socket) => { // on a connection... perform the following.
     console.log('a user connected');
     users[socket.id] = {
         ID : socket.id, // adds to a user directory, the queue
     }
     currUser = Object.keys(users)[0]; // checks current user
     io.emit('waitlist-update', Object.keys(users).length-1); // updates the waitlist

     socket.on('connect-to-bot', () => {
         console.log("sent out"); // on a connection to the bot, send this
         if(socket.id == currUser){
             io.broadcast.to(socket.id).emit('verified');
         }
         else{
             io.broadcast.to(socket.id).emit('denied');
         }
     })

     socket.on('up', () => {
         console.log("Moving bot up"); // moving bot up
     });

     socket.on('left', () => {
         console.log("Moving bot left"); // moving bot left
     });

     socket.on('right', () => {
         console.log("Moving bot right"); // moving bot right
     });

     socket.on('down', () => {
         console.log("Moving bot down"); // moving both down
     });

     socket.on('disconnect', () => {
         console.log('user disconnected'); // on disconnect, delete everything about that user
         delete users[socket.id];
         io.emit('waitlist-update', Object.keys(users).length-1);
     });
   });
```

#### Client Side -- Attempting a connection

This was the first connection before getting to the connection

```javascript
function connect() { // when connecting to the big button
             socket.emit("connect-to-bot");
         }
         socket.on("verified", ()=>{ // when connection verified, move over
             alert('Connecting...');   
             window.location.assign(window.location.origin+"/controller.html");
         });
         socket.on("denied", ()=>{ // When denied, wait
             alert("unable to connect! Please wait until space frees up!");
         });
         socket.on("waitlist-update", (length)=>{
             console.log("occured"); // Checks updates to the waitlist
             document.getElementById("waitlistCount").innerText = length;
         })
```

#### Other Client Side Function: 

This used a function (as seen with the `case` based functions). Works whenever mouse buttons are pressed down.


```javascript
 var socket = io();

 /*
 Works when mouse buttons pressed down
 */
const u = document.getElementById("up");
const l = document.getElementById("left");
const r = document.getElementById("right");
const d = document.getElementById("down");
u.addEventListener('mousedown', () => {
    isButtonPressed = true;
    timerId = setInterval(function(){emitSocket("up");}, 100); // Adjust the interval as needed
});
u.addEventListener('mouseup', () => {
    isButtonPressed = false;
    clearInterval(timerId);
});
l.addEventListener('mousedown', () => {
    isButtonPressed = true;
    timerId = setInterval(function(){emitSocket("left");}, 100); // Adjust the interval as needed
});
l.addEventListener('mouseup', () => {
    isButtonPressed = false;
    clearInterval(timerId);
});
r.addEventListener('mousedown', () => {
    isButtonPressed = true;
    timerId = setInterval(function(){emitSocket("right");}, 100); // Adjust the interval as needed
});
r.addEventListener('mouseup', () => {
    isButtonPressed = false;
    clearInterval(timerId);
});
d.addEventListener('mousedown', () => { 
    isButtonPressed = true;
    timerId = setInterval(function(){emitSocket("down");}, 100); // Adjust the interval as needed
});
d.addEventListener('mouseup', () => {
    isButtonPressed = false;
    clearInterval(timerId);
});

function emitSocket(dir) {
    if (isButtonPressed) {
        switch(dir){
            case "up":
                socket.emit("up");
                break;
            case "left":
                socket.emit("left");
                break;
            case "right":
                socket.emit("right");
                break;
            case "down":
                socket.emit("down");
                break;
        }

    }
}
```

## The Third Architecture: Using Java

This server side architecture used Java in order to take functions from a `MotorFunctions.java` file to call each function. This was planned to be hosted on the raspberry pi and then the server side deployed elsewhere.

In [1]:
package com.corundumstudio.socketio.demo;

import com.corundumstudio.socketio.listener.*;

import java.util.HashSet;
import java.util.Set;

import com.corundumstudio.socketio.*;


public class MotorSocketSetup {

        public static Set<String> userQueue = new HashSet<>();

    public static void main(String[] args) throws InterruptedException {

        Configuration config = new Configuration();
        config.setHostname("localhost");
        config.setPort(9092);

        final SocketIOServer server = new SocketIOServer(config);

        // probably have to change `String.class` to `Integer.class` I'd check if ID is a integer or stinrg
        server.addEventListener("checkUser", String.class, new DataListener<String>(){
            @Override
            public void onData(final SocketIOClient client, String userId, final AckRequest ackRequest){
                boolean userExists = userQueue.contains(userId);
                client.sendEvent("checkExist", userExists);
                System.out.printf("User with id %s exists "+userId);

            }
        });


        server.addConnectListener(new ConnectListener() {
            @Override
            public void onConnect(SocketIOClient client) {
                String userId = client.getSessionId().toString();
                userQueue.add(userId);
                server.getBroadcastOperations().sendEvent("newUserJoined", "New user joined with ID: "+userId);
                System.out.println("New user joined with ID: " + userId);
            }
            
        });
        server.addEventListener("up", String.class, new DataListener<String>() {
            @Override
            public void onData(final SocketIOClient client, String data, final AckRequest ackRequest) {
                // Run code to go up
                MotorFunctions.up();
                System.out.println("going up");
        
            }
        });

        server.addEventListener("left", String.class, new DataListener<String>() {
            @Override
            public void onData(final SocketIOClient client, String data, final AckRequest ackRequest) {
                // run code to go left
                MotorFunctions.left();
                System.out.println("going left");
            }
        });

        server.addEventListener("right", String.class, new DataListener<String>() {
            @Override
            public void onData(final SocketIOClient client, String data, final AckRequest ackRequest) {
                // run code to go right
                MotorFunctions.right();
                System.out.println("going right");

            }
        });


        server.addEventListener("down", String.class, new DataListener<String>() {
            @Override
            public void onData(final SocketIOClient client, String data, final AckRequest ackRequest) {
                // run code to go down
                MotorFunctions.down();
                System.out.println("going down");
    
            }
        });



        server.start();

        Thread.sleep(Integer.MAX_VALUE);

        server.stop();
    }

}

Print


#### MotorFunctions.java

each of these would be completed and then run up. Code for this was not implemented due to issues with the drivetrain.

In [ ]:
package com.corundumstudio.socketio.demo;

 public class MotorFunctions {

     public static void up(){
         System.out.println("Moving Up via function");
     }
     public static void left(){
         System.out.println("Moving Up via function");
     }
     public static void right(){
         System.out.println("Moving Up via function");
     }
     public static void down(){
         System.out.println("Moving Up via function");
     }
 }

# Analytics

Showing relative commits
![Analytics1](https://media.discordapp.net/attachments/981743033496256562/1170921979218513940/Analytics1.png?ex=655acd12&is=65485812&hm=65f00417cc9ca0566d78c133b2fe1a6ab780638ab1856deda6d62766e1a08a8c&=&width=1621&height=1060)

Also showing relative commits
![Analytics2](https://cdn.discordapp.com/attachments/981743033496256562/1170921979524690012/Analytics2.png?ex=655acd12&is=65485812&hm=d3a142b5e79c5d8d7654ffee1938323d8c77048db7d76d17edc86f7eea9e1bc5&)

Showing code/commit freq.
![Analytics3](https://cdn.discordapp.com/attachments/981743033496256562/1170921979877003355/Analytics3.png?ex=655acd12&is=65485812&hm=d1cf305710b74b9cf56e88aa4eaba505db1c5ad854f0e4568b4e09a54603e74f&)

Total contributions
![Analytics4](https://cdn.discordapp.com/attachments/981743033496256562/1170921980141260951/Analytics4.png?ex=655acd12&is=65485812&hm=fc9f5e168fcea88fced6f38af352a3ccf1aa9f0ee3dde6ccaa47c1ac720b3c79&)